<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=07bc92e667a66a1fa8a17419c6063d8f7504352c2a5f9833c7c7e81891a0f6cc
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-22 11:49:33
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: 96.27 K (0.69%)
Current PnL: -25.83 L (-16.69%)
CY Booked + Current PnL: -11.54 L (-7.46%)
-------------------
Total profit:  1.59 L
Total loss:  -27.42 L
-------------------
Total Booked + Current PnL: 15.44 L (12.08%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.12%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.47 L (64.06%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.51%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.60,-19.15,23.80,0.09,19391.0,-19302.0,81475.0,89.11,35.0,M-SC,2.28,253.0,-1.00,0.58,7.04,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.79,8.45,10.37,19.69,21564.0,16203.0,207945.0,-2.18,54.0,X-MC,2.46,78.0,0.75,1.48,15.82,XY25,NTT,AC
50,MASFIN,398.61,-1.12,-4.35,27.60,22.05,25867.0,-4260.0,93720.0,-17.34,48.0,H-SC,6.43,164.0,-0.16,0.67,36.23,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-0.13,-15.68,18.62,0.02,27406.0,-27371.0,147188.0,124.33,44.0,M-SC,12.63,234.0,-1.00,1.05,28.03,OX40N,NTT,PAINTS
43,ITC,452.00,0.42,-1.29,12.23,10.78,29012.0,-3098.0,237220.0,-41.15,49.0,X-LC,2.16,5.0,-0.11,1.69,3.88,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.0,9.75,-56.67,130.95,0.07,77833.0,-77740.0,59437.0,2078.03,76.0,L-SC,3.03,268.0,-1.00,0.42,47.82,OX40N,NTT,JEWELLERY
21,DEN,75.0,3.23,-38.89,132.49,42.07,68386.0,-32848.0,51616.0,83.94,56.0,M-SC,7.81,235.0,-0.48,0.37,9.62,AR,NTT,ENTERTAINMENT
84,WIPRO,420.0,3.17,11.90,53.95,72.27,102295.0,20162.0,189610.0,-3.81,77.0,M-LC,3.38,99.0,0.20,1.35,18.81,XR,NTT,IT
13,BSOFT,831.7,2.78,-19.23,86.69,50.79,98482.0,-27051.0,113602.0,3.59,70.0,H-SC,7.21,151.0,-0.27,0.81,32.27,XR,ATH,IT
28,GREENPANEL,537.0,2.53,-37.91,130.55,43.14,148390.0,-69413.0,113665.0,154.52,37.0,M-SC,10.09,240.0,-0.47,0.81,6.92,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JCHAC,2282.00,-2.27,-36.53,57.54,-0.01,46674.0,-46689.0,81116.0,16508.70,33.0,M-SC,10.48,233.0,-1.00,0.58,3.82,OX40N,NTT,AC
18,COFFEEDAY,80.00,-1.77,-40.85,128.70,35.27,86437.0,-46387.0,67162.0,-53.72,48.0,L-SC,10.04,270.0,-0.54,0.48,64.38,XR,NTT,HOTELS
51,MEDANTA,1486.00,-1.45,-3.75,28.91,24.08,35324.0,-4764.0,122186.0,-11.59,39.0,X-SC,5.93,89.0,-0.13,0.87,15.13,XY24,NTT,HEALTHCARE
50,MASFIN,398.61,-1.12,-4.35,27.60,22.05,25867.0,-4260.0,93720.0,-17.34,48.0,H-SC,6.43,164.0,-0.16,0.67,36.23,XR,ATH,FINANCE
8,AWL,485.00,-0.96,-23.25,100.00,53.50,231588.0,-70163.0,231588.0,-62.86,35.0,X-MC,4.79,58.0,-0.30,1.65,2.86,XY24,NTT,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.48,0.18,107.53,107.91,159055.0,273.0,147917.0,-18.52,62.0,M-SC,12.01,216.0,0.0,1.05,10.29,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.48,0.18,107.53,107.91,159055.0,273.0,147917.0,-18.52,62.0,M-SC,12.01,216.0,0.00,1.05,10.29,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.13,-15.68,18.62,0.02,27406.0,-27371.0,147188.0,124.33,44.0,M-SC,12.63,234.0,-1.00,1.05,28.03,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.60,-19.15,23.80,0.09,19391.0,-19302.0,81475.0,89.11,35.0,M-SC,2.28,253.0,-1.00,0.58,7.04,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,1.21,-21.04,67.20,32.03,69956.0,-27734.0,104101.0,-14.53,65.0,H-SC,9.32,148.0,-0.40,0.74,16.60,OX40N,NTT,IT
66,SIS,528.00,-0.23,-23.72,58.96,21.26,49938.0,-26334.0,84698.0,1998.96,52.0,H-SC,4.73,166.0,-0.53,0.60,14.44,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,3.17,11.90,53.95,72.27,102295.0,20162.0,189610.0,-3.81,77.0,M-LC,3.38,99.0,0.2,1.35,18.81,XR,NTT,IT
25,FINCABLES,1641.55,0.48,0.18,107.53,107.91,159055.0,273.0,147917.0,-18.52,62.0,M-SC,12.01,216.0,0.0,1.05,10.29,OX40N,ATH,CABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,3.17,11.90,53.95,72.27,102295.0,20162.0,189610.0,-3.81,77.0,M-LC,3.38,99.0,0.20,1.35,18.81,XR,NTT,IT
25,FINCABLES,1641.55,0.48,0.18,107.53,107.91,159055.0,273.0,147917.0,-18.52,62.0,M-SC,12.01,216.0,0.00,1.05,10.29,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-0.21,-4.95,117.49,106.72,137735.0,-6105.0,117231.0,-52.92,36.0,H-SC,3.97,139.0,-0.04,0.83,17.27,AR,ATH,MISC
6,ASIANTILES,137.00,-0.35,-4.79,90.81,81.67,81111.0,-4491.0,89319.0,8038.89,70.0,L-SC,11.53,271.0,-0.06,0.64,72.97,XR,NTT,CERAMICS
85,ZYDUSLIFE,1286.17,0.48,-2.56,39.20,35.63,80048.0,-5370.0,204204.0,-16.30,44.0,H-MC,2.50,119.0,-0.07,1.45,13.68,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,0.67,-17.81,89.16,55.47,178284.0,-43326.0,199960.0,-27.18,32.0,X-MC,14.80,64.0,-0.24,1.42,2.46,X40N,ATH,FINANCE
8,AWL,485.00,-0.96,-23.25,100.00,53.50,231588.0,-70163.0,231588.0,-62.86,35.0,X-MC,4.79,58.0,-0.30,1.65,2.86,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.13,-38.76,120.77,35.20,95167.0,-49870.0,78800.0,3.25,35.0,X-SC,16.76,92.0,-0.52,0.56,1.08,X40,NTT,FOOTWEAR
33,HINDUNILVR,2922.00,0.25,-11.09,27.84,13.67,63631.0,-28503.0,228560.0,-17.78,35.0,X-LC,2.71,14.0,-0.45,1.63,7.90,XY25,NTT,FMCG
34,HONAUT,58357.33,-0.34,-17.31,73.32,43.32,98747.0,-28192.0,134680.0,-28.92,35.0,X-SC,5.42,90.0,-0.29,0.96,3.41,X40N,ATH,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,0.77,0.09,24.36,24.48,61408.0,231.0,252087.0,-80.87,50.0,X-SC,0.92,86.0,0.00,1.79,23.25,X40N,NTT,MISC
12,BERGEPAINT,680.0,1.38,-2.83,24.67,21.14,54498.0,-6432.0,220907.0,-9.65,47.0,X-MC,1.15,74.0,-0.12,1.57,24.51,XY24,NTT,PAINTS
20,DABUR,735.0,-0.01,-3.44,48.82,43.70,116944.0,-8526.0,239542.0,-11.44,36.0,X-MC,2.05,72.0,-0.07,1.70,11.10,XY24,BTT,FMCG
78,UNITDSPR,1644.0,0.29,3.64,16.53,20.77,40111.0,8514.0,242658.0,-5.34,46.0,X-MC,2.12,62.0,0.21,1.73,10.81,X40N,NTT,BREWERIES
43,ITC,452.0,0.42,-1.29,12.23,10.78,29012.0,-3098.0,237220.0,-41.15,49.0,X-LC,2.16,5.0,-0.11,1.69,3.88,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COLPAL,3726.84,-0.74,-20.43,77.85,41.51,163135.0,-53815.0,209550.0,-5.20,38.0,X-MC,7.66,61.0,-0.33,1.49,0.39,XY25,ATH,FMCG
3,ACC,3906.00,0.98,-25.67,120.85,64.15,213735.0,-61091.0,176860.0,-56.31,38.0,X-SC,3.61,82.0,-0.29,1.26,0.98,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.13,-38.76,120.77,35.20,95167.0,-49870.0,78800.0,3.25,35.0,X-SC,16.76,92.0,-0.52,0.56,1.08,X40,NTT,FOOTWEAR
59,RELAXO,1176.00,0.26,-47.78,191.67,52.31,145290.0,-69358.0,75802.0,-44.44,42.0,X-SC,7.29,91.0,-0.48,0.54,1.46,X40N,NTT,FOOTWEAR
53,PAGEIND,51605.07,1.92,-9.56,41.66,28.11,75884.0,-19260.0,182150.0,-34.50,36.0,X-MC,14.03,55.0,-0.25,1.30,2.06,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.84,-29.00,101.18,42.84,46701.0,-18850.0,46156.0,-53.72,47.0,X-SC,34.56,83.0,-0.40,0.33,4.42,XY24,NTT,MISC
59,RELAXO,1176.00,0.26,-47.78,191.67,52.31,145290.0,-69358.0,75802.0,-44.44,42.0,X-SC,7.29,91.0,-0.48,0.54,1.46,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.13,-38.76,120.77,35.20,95167.0,-49870.0,78800.0,3.25,35.0,X-SC,16.76,92.0,-0.52,0.56,1.08,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-1.45,-3.75,28.91,24.08,35324.0,-4764.0,122186.0,-11.59,39.0,X-SC,5.93,89.0,-0.13,0.87,15.13,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.34,-17.31,73.32,43.32,98747.0,-28192.0,134680.0,-28.92,35.0,X-SC,5.42,90.0,-0.29,0.96,3.41,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.88,-8.96,32.59,20.71,112222.0,-33893.0,344344.0,-21.68,72.0,X-LC,3.66,1.0,-0.30,2.45,15.06,X40,ATH,IT
41,INFY,1972.00,2.34,14.17,17.58,34.24,62800.0,44334.0,357222.0,-10.98,74.0,X-LC,5.31,2.0,0.71,2.54,23.63,X40,NTT,IT
75,TMPV,600.00,2.23,-26.11,66.44,22.98,110896.0,-58986.0,166912.0,-23.08,51.0,X-LC,6.54,3.0,-0.53,1.19,4.98,XY24,NTT,AUTO
81,VBL,671.64,2.34,-2.96,39.81,35.67,122016.0,-9347.0,306495.0,-14.52,57.0,X-LC,3.79,4.0,-0.08,2.18,10.49,X40N,ATH,FMCG
43,ITC,452.00,0.42,-1.29,12.23,10.78,29012.0,-3098.0,237220.0,-41.15,49.0,X-LC,2.16,5.0,-0.11,1.69,3.88,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.70,-36.58,109.53,32.89,53632.0,-28240.0,48966.0,-704.67,59.0,L-MC,6.63,259.0,-0.53,0.35,34.85,XR,NTT,BANKS
56,RAJESHEXPO,518.00,9.75,-56.67,130.95,0.07,77833.0,-77740.0,59437.0,2078.03,76.0,L-SC,3.03,268.0,-1.00,0.42,47.82,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-0.35,-4.79,90.81,81.67,81111.0,-4491.0,89319.0,8038.89,70.0,L-SC,11.53,271.0,-0.06,0.64,72.97,XR,NTT,CERAMICS
50,MASFIN,398.61,-1.12,-4.35,27.60,22.05,25867.0,-4260.0,93720.0,-17.34,48.0,H-SC,6.43,164.0,-0.16,0.67,36.23,XR,ATH,FINANCE
13,BSOFT,831.70,2.78,-19.23,86.69,50.79,98482.0,-27051.0,113602.0,3.59,70.0,H-SC,7.21,151.0,-0.27,0.81,32.27,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,9.75,-56.67,130.95,0.07,77833.0,-77740.0,59437.0,2078.03,76.0,L-SC,3.03,268.0,-1.00,0.42,47.82,OX40N,NTT,JEWELLERY
13,BSOFT,831.70,2.78,-19.23,86.69,50.79,98482.0,-27051.0,113602.0,3.59,70.0,H-SC,7.21,151.0,-0.27,0.81,32.27,XR,ATH,IT
61,ROUTE,2227.21,-0.25,-45.49,210.15,69.05,152418.0,-60538.0,72528.0,-55.87,56.0,H-SC,21.34,140.0,-0.40,0.52,11.85,SR,ATH,IT
84,WIPRO,420.00,3.17,11.90,53.95,72.27,102295.0,20162.0,189610.0,-3.81,77.0,M-LC,3.38,99.0,0.20,1.35,18.81,XR,NTT,IT
6,ASIANTILES,137.00,-0.35,-4.79,90.81,81.67,81111.0,-4491.0,89319.0,8038.89,70.0,L-SC,11.53,271.0,-0.06,0.64,72.97,XR,NTT,CERAMICS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.43
1,25,44.85
2,50,76.59


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.92
MC    30.79
LC    24.31
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.76
X40      23.09
X40N     12.98
XR        9.86
XY25      9.03
AR        8.93
OX40N     7.60
SR        1.01
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.28
H-SC    23.67
X-LC    20.84
M-SC    11.69
X-SC     8.02
H-MC     4.81
L-SC     1.54
M-LC     1.35
M-MC     1.35
H-LC     1.11
L-LC     1.01
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.23,-7.69,42.32
IT,13.22,-13.14,70.03
FINANCE,9.46,-19.84,71.96
MISC,7.28,-26.96,78.89
ELECTRICAL,6.00,-11.45,52.27
PAINTS,5.82,-13.05,29.85
INSURANCE,4.74,-1.59,36.36
PHARMA,4.09,-2.12,34.27
AUTO,2.87,-30.27,75.43


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3162943.0,21
AR,1316841.0,10
XR,1277954.0,13
X40,978209.0,14
X40N,860936.0,9
OX40N,722772.0,10
XY25,371305.0,6
SR,278345.0,2
MH,77527.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3650475.0,25
M-SC,1418439.0,15
X-MC,1413856.0,16
X-LC,797363.0,11
X-SC,792276.0,8
H-MC,403444.0,3
L-SC,245381.0,3
M-LC,102295.0,1
H-LC,77315.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1275786.0      6
           AR         905116.0      5
M-SC       XY24       809699.0      6
H-SC       XR         781077.0      7
X-MC       X40        485224.0      7
           XY24       403030.0      3
           X40N       340903.0      4
H-SC       OX40N      332624.0      4
M-SC       OX40N      312315.0      5
X-SC       X40N       305445.0      3
X-LC       X40        301914.0      5
X-SC       XY24       295760.0      3
H-SC       SR         278345.0      2
X-LC       X40N       214588.0      2
H-MC       AR         211387.0      2
           XY24       192057.0      1
X-SC       X40        191071.0      2
X-LC       XY24       186611.0      2
X-MC       XY25       184699.0      2
M-SC       XR         173402.0      2
L-SC       XR         167548.0      2
M-SC       AR         123023.0      2
M-LC       XR         102295.0      1
X-LC       XY25        94250.0      2
L-SC       OX40N       77833.0      1
H-SC       MH          77527.0      1
H-LC       AR          77315.0      1
M-MC       XY25        55499.0      1
L-MC       XR          53632.0      1
L-LC       XY25        36857.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
